### IMPORTAÇÃO DAS BIBLIOTECAS

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import plotly.io as pio
import os

### CONEXÃO COM O BANCO DE DADOS MYSQL

In [2]:
conn = sqlite3.connect("dados_turmas.db")
cursor = conn.cursor()
query = "SELECT * FROM alunos"
df = pd.read_sql_query(query, conn)
conn.close()

### VERIFICANDO A CONSISTÊNCIA DOS DADOS

In [3]:
# Renomenado as colunas

colunas_renomear = {"id":"id", "nome":"Nome","data_nasc":"Nascimento","turma":"Turma","bim":"Bimestre","avaliacao_1":"Inst_1", "avaliacao_2":"Inst_2","avaliacao_3":"Inst_3", "avaliacao_4":"Comportamento","avaliacao_5":"CAED"}

df.rename(columns=colunas_renomear, inplace=True)

In [4]:
# Transformando os dados em numéricos

cols = ['Inst_1',"Inst_2","Inst_3","Comportamento","CAED","Ano"]

for col in cols:
    df[col] = pd.to_numeric(df[col], errors = "coerce")

In [5]:
# Inserindo a coluna idade

hoje = datetime.now()
df['Nascimento'] = pd.to_datetime(df["Nascimento"], dayfirst=True, format=r"%d/%m/%Y")
df["Idade"] = df.Nascimento.apply(lambda x: hoje.year - x.year - ((hoje.month,hoje.day) < (x.month,x.day)))
df.fillna(0)

,id,Nome,Nascimento,Turma,Bimestre,Inst_1,Inst_2,Inst_3,Comportamento,CAED,Ano,Idade
0,1,ANA CLARA VIEIRA,2009-08-18,900,4,9.0,14.0,30.0,20.0,8.0,2024,15
1,2,CLARA SCHULTZ,2010-07-29,900,4,18.0,14.0,0.0,10.0,12.0,2024,14
2,3,EMANUELLY CRISTINA,2010-03-08,900,4,23.0,12.0,30.0,20.0,21.0,2024,14
3,4,EMILLY PEDRO,2010-08-16,900,4,2.0,14.0,0.0,5.0,10.0,2024,14
4,5,ERICK JUNIOR,2008-11-05,900,4,0.0,0.0,0.0,5.0,8.0,2024,16
...,...,...,...,...,...,...,...,...,...,...,...,...
66,67,PAULO BERNARDO,2010-06-11,902,4,6.0,14.0,20.0,0.0,8.0,2024,14
67,68,RAPHAEL FERREIRA,2010-06-11,902,4,11.0,14.0,30.0,20.0,11.0,2024,14
68,69,RAPHAEL RODOVALHO,2010-06-11,902,4,13.0,10.0,30.0,20.0,11.0,2024,14
69,70,VICTORIA MARQUES,2010-06-11,901,4,30.0,12.0,25.0,20.0,11.0,2024,14


In [6]:
# Separando o dataframe por turma

df_900 = df.query("Turma=='900'").sort_values(by='Nome', ascending=False)
df_901 = df.query("Turma=='901'").sort_values(by='Nome', ascending=False)
df_902 = df.query("Turma=='902'").sort_values(by='Nome', ascending=False)

### GRÁFICOS

In [7]:
fig1 = px.bar(
    df_900,
    y="Nome",
    x = ["Inst_1","Inst_2","Inst_3","Comportamento","CAED"],
    orientation = "h",
    labels={"variable": "Instrumentos Avaliativos"},
    text_auto=True
)

fig2 = px.bar(
    df_901,
    y="Nome",
    x = ["Inst_1","Inst_2","Inst_3","Comportamento","CAED"],
    orientation = "h",
    labels={"variable": "Instrumentos Avaliativos"},
    text_auto=True
)

fig3 = px.bar(
    df_902,
    y="Nome",
    x = ["Inst_1","Inst_2","Inst_3","Comportamento","CAED"],
    orientation = "h",
     labels={"variable": "Instrumentos Avaliativos"},
     text_auto=True
)

def atualizar_layout(fig, turma:str):
    fig.update_layout(
        title={
            "text": f"Turma {str(turma)} - Notas dos Instrumentos Avaliativos",
            "y": 0.98,
            "x": 0.5,
            "xanchor": "center",
            "yanchor":"top",
            "font":{"size":24}
        },
    xaxis_title = "notas",
    xaxis_range=[0,100],
    xaxis = dict(
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis = dict(
        title_font = dict(size=18),
        tickfont=dict(size=14)
    ),

    width=1500,
    height = 700
    )

atualizar_layout(fig1,900)
atualizar_layout(fig2,901)
atualizar_layout(fig3,902)

#fig1.show()
#fig2.show()
#fig3.show()


### CÁLCULO DAS MÉDIAS POR TURMA

In [8]:
#Media de notas por turnma

df_mean = df.groupby("Turma")[['Inst_1','Inst_2','Inst_3','CAED','Comportamento']].mean()

fig4 = px.bar(
    df_mean,
    y=['Inst_1','Inst_2','Inst_3','CAED','Comportamento'],
    labels={"variable": "Instrumentos Avaliativos"},
    text_auto=True,
    )

fig4.update_layout(
        title={
        'text': 'MÉDIA DAS AVALIAÇÕES POR TURMA',  # Título do gráfico
        'y': 0.96,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24}  # Tamanho da fonte do título
    },
    xaxis_title='Turma',  # Nome do eixo x
    xaxis=dict(
        title_font=dict(size=18),  # Tamanho da fonte do título do eixo x
        tickfont=dict(size=14)  # Tamanho da fonte dos ticks do eixo x
    ),
    yaxis_title="Notas",
    yaxis=dict(
        title_font=dict(size=18),  # Tamanho da fonte do título do eixo y
        tickfont=dict(size=14)  # Tamanho da fonte dos ticks do eixo y
    ),
    width=1500,
    height=700
)
#fig4.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Instrumentos Avaliativos=Inst_1<br>Turma=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Inst_1',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Inst_1',
              'offsetgroup': 'Inst_1',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'texttemplate': '%{y}',
              'type': 'bar',
              'x': array(['900', '901', '902'], dtype=object),
              'xaxis': 'x',
              'y': array([10.7826087 , 20.        , 11.78947368]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'Instrumentos Avaliativos=Inst_2<br>Turma=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Inst_2',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Inst_2',
              'offsetgroup': 'Inst_2',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'texttemplate': '%{y}',
              'type': 'bar',
              'x': array(['900', '901', '902'], dtype=object),
              'xaxis': 'x',
              'y': array([11.47826087, 11.48275862, 11.73684211]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'Instrumentos Avaliativos=Inst_3<br>Turma=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Inst_3',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': 'Inst_3',
              'offsetgroup': 'Inst_3',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'texttemplate': '%{y}',
              'type': 'bar',
              'x': array(['900', '901', '902'], dtype=object),
              'xaxis': 'x',
              'y': array([ 6.52173913, 23.79310345, 24.15789474]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'Instrumentos Avaliativos=CAED<br>Turma=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'CAED',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': 'CAED',
              'offsetgroup': 'CAED',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'texttemplate': '%{y}',
              'type': 'bar',
              'x': array(['900', '901', '902'], dtype=object),
              'xaxis': 'x',
              'y': array([ 9.86956522, 10.82758621, 10.15789474]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Instrumentos Avaliativos=Compo' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'Comportamento',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': 'Comportamento',
              'offsetgroup': 'Comportamento',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'texttemplate': '%{y}',
              'type': 'bar',
              'x': array(['900', '901', '902'], dtype=object),
              'xaxis': 'x',
              'y': array([15.2173913, 17.4137931, 14.       ]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'height': 700,
               'legend': {'title': {'text': 'Instrumentos Avaliativos'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'title': {'font': {'size': 24},
                         'text': 'MÉDIA DAS AVALIAÇÕES POR TURMA',
                         'x': 0.5,
                         'xanchor': 'center',
                         'y': 0.96,
                         'yanchor': 'top'},
               'width':

### GERAR OS HTMLS PLOTLY

In [9]:
#Gerando os HTMLS

path_imgs = "imagens/"

if not os.path.exists(path_imgs):
    os.makedirs(path_imgs)
    print(f"Pasta {path_imgs} criada!")
else:
    print(f"Pasta {path_imgs} já existe. Não foi realizada nenhuma ação.")

figs = [('900',fig1),('901',fig2),('902',fig3),('media',fig4)]

for turma, fig in figs:
    pio.write_html(fig,f"imagens/notas_{turma}.html", auto_open=False)

Pasta imagens/ já existe. Não foi realizada nenhuma ação.


### GRÁFICOS MATPLOTLIB

In [10]:
def graficos_turma(df, turma: str, output_path: str = "imagens", xlim: int = 100):
    """
    Creates a stacked horizontal bar chart summarizing evaluations for a specific class (turma).

    Parameters:
        df (pd.DataFrame): Data containing columns 'Nome', 'Inst_1', 'Inst_2', etc.
        turma (str): Name of the class (Turma).
        output_path (str): Directory where the chart will be saved.
        xlim (int): Maximum x-axis value for the chart.
    """
    # Define metrics to include in the chart
    metrics = ['Inst_1', 'Inst_2', 'Inst_3', 'Comportamento', 'CAED']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']  # Optional color scheme

    # Ensure required columns exist
    missing_cols = [col for col in metrics if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in DataFrame: {missing_cols}")

    # Create figure and axis
    fig, ax = plt.subplots(1, 1, figsize=(15, 7))

    # Initialize starting position for stacking
    left_positions = [0] * len(df)

    # Add each metric to the bar chart
    for metric, color in zip(metrics, colors):
        hbar = ax.barh(
            df['Nome'], width=df[metric], left=left_positions, label=metric, color=color
        )
        # Add labels
        ax.bar_label(
            hbar, labels=[f"{v:.2f}" if v != 0 else '' for v in hbar.datavalues], fontsize=10
        )
        # Update left positions for the next metric
        left_positions += df[metric]

    # Chart styling
    ax.set_title(f"Instrumentos Avaliativos - Turma {turma}", fontdict={'size': 26})
    ax.set_xlabel("Notas", fontsize=14)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=len(metrics), fontsize=14)
    ax.grid(True, axis='x')
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.set_xlim(0, xlim)

    # Save the chart
    plt.tight_layout()
    output_file = f"{output_path}/notas_{turma}.png"
    fig.savefig(output_file, format="png", dpi=500)
    plt.close(fig)
    print(f"Chart saved to: {output_file}")


In [11]:
graficos_turma(df_900,"900",xlim=110)
graficos_turma(df_901,"901", xlim=110)
graficos_turma(df_902,"902", xlim=110)

Chart saved to: imagens/notas_900.png
Chart saved to: imagens/notas_901.png
Chart saved to: imagens/notas_902.png


### GRÁFICOS MÉDIA MATPLOTLIB

In [12]:
def plot_mean_scores(df_mean, output_path="imagens/medias.png"):
    """
    Generates a stacked bar chart displaying the mean scores of evaluative metrics per Turma.

    Parameters:
        df_mean (pd.DataFrame): DataFrame with mean scores (columns like 'Inst_1', 'Inst_2', etc.) indexed by 'Turma'.
        output_path (str): Filepath to save the chart as a PNG.
    """
    # Define metrics dynamically
    metrics = ['Inst_1', 'Inst_2', 'Inst_3', 'Comportamento', 'CAED']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']  # Optional color scheme

    # Ensure required columns exist
    missing_cols = [col for col in metrics if col not in df_mean.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in DataFrame: {missing_cols}")

    # Prepare figure and axis
    fig_media, ax_media = plt.subplots(1, 1, figsize=(15, 7))

    # X-axis categories
    x = list(df_mean.index)  # Assuming 'Turma' as the index

    # Initialize stacking position
    bottoms = np.zeros(len(df_mean))

    # Create stacked bars dynamically
    for metric, color in zip(metrics, colors):
        bars = ax_media.bar(
            x, df_mean[metric], bottom=bottoms, label=metric, color=color
        )
        # Add bar labels
        ax_media.bar_label(
            bars, labels=[f"{v:.2f}" if v > 0 else '' for v in bars.datavalues], fontsize=10
        )
        # Update stacking positions
        bottoms += df_mean[metric]

    # Customize chart
    ax_media.set_title("Média dos Instrumentos Avaliativos por Turma", fontdict={'size': 26})
    ax_media.set_xlabel("Turma", fontsize=14)
    ax_media.set_ylabel("Notas", fontsize=14)
    ax_media.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=len(metrics), fontsize=14)
    ax_media.tick_params(axis='x', labelsize=12)
    ax_media.tick_params(axis='y', labelsize=12)
    ax_media.grid(axis='y', linestyle='--', alpha=0.7)

    # Adjust layout and save
    plt.tight_layout()
    fig_media.savefig(output_path, format="png", dpi=500)
    plt.close(fig_media)
    print(f"Chart saved to: {output_path}")

plot_mean_scores(df_mean)



Chart saved to: imagens/medias.png


### GERANDO O SOMATORIO DAS NOTAS DE CADA TURMA

In [13]:
df_900['total'] = df_900.Inst_1 + df_900.Inst_2 + df_900.Inst_3 + df_900.Comportamento + df_900.CAED
df_901['total'] = df_901.Inst_1 + df_901.Inst_2 + df_901.Inst_3 + df_901.Comportamento + df_901.CAED
df_902['total'] = df_902.Inst_1 + df_902.Inst_2 + df_902.Inst_3 + df_902.Comportamento + df_902.CAED
df['total'] = df.Inst_1 + df.Inst_2 + df.Inst_3 + df.Comportamento + df.CAED

GRÁFICOS DAS NOTAS TOTAIS

In [14]:
def graficos_total(cor=[], df=[]):

    fig_total, ax_total = plt.subplots(3,1, figsize=(15,20))

    for linha in range(3):
        total = ax_total[linha].barh(df[linha]['Nome'], width=df[linha]['total'], color=cor[linha], label=f"Total dos Inst. Avaliativos - 90{linha}")
        ax_total[linha].bar_label(total)
        ax_total[linha].set_xlim(0,100)
        ax_total[linha].grid(True, axis="x")
        ax_total[linha].legend()
        ax_total[linha].set_title(f"TOTAL DOS INSTRUMENTOS AVALIATIVOS - Turma 90{str(linha)} - 4 Bim")
        fig_total.savefig("imagens/notas_totais.png")
        fig_total.tight_layout()
        plt.close(fig_total)

graficos_total(['blue','green','black'],[df_900, df_901, df_902])

### CRIANDO POWERPOINT

In [15]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

# Criar a apresentação
prs = Presentation()

# Configurar o tamanho do slide para widescreen (16:9)
prs.slide_width = Inches(13.33)  # Largura em widescreen
prs.slide_height = Inches(7.5)  # Altura em widescreen

# Slide 1: Slide de título
slide_layout = prs.slide_layouts[6]  # Layout em branco (para controle total)
slide = prs.slides.add_slide(slide_layout)

# Adicionar título
title_box = slide.shapes.add_textbox(Inches(1), Inches(2), Inches(11.33), Inches(1.5))
title_frame = title_box.text_frame
title_frame.text = "Notas das Turmas"
title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Centralizar texto
title_frame.paragraphs[0].font.size = Pt(60)  # Ajustar tamanho da fonte

# Adicionar subtítulo
subtitle_box = slide.shapes.add_textbox(Inches(1), Inches(4), Inches(11.33), Inches(1))
subtitle_frame = subtitle_box.text_frame
subtitle_frame.text = "4º bimestre - E. M. A. Pereira Bruno"
subtitle_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Centralizar texto
subtitle_frame.paragraphs[0].font.size = Pt(44)  # Ajustar tamanho da fonte

# Imagens (Substitua os caminhos abaixo pelos arquivos das suas imagens)
imagens = [
    "imagens/notas_900.png",
    "imagens/notas_901.png",
    "imagens/notas_902.png",
    "imagens/medias.png"
]

# Adicionar os slides com imagens
for img_path in imagens:
    slide_layout = prs.slide_layouts[6]  # Layout em branco
    slide = prs.slides.add_slide(slide_layout)

    # Definir a posição e o tamanho da imagem para cobrir todo o slide
    left = 0  # Margem esquerda
    top = 0   # Margem superior
    slide.shapes.add_picture(img_path, left, top, width=prs.slide_width, height=prs.slide_height)

# Salvar a apresentação
prs.save("apresentacao_widescreen.pptx")

# Abrir o arquivo automaticamente
file_path = "apresentacao_widescreen.pptx"
#os.startfile(file_path)

print("Apresentação criada com sucesso!")


Apresentação criada com sucesso!


### GERANDO O EXCEL COM OS DADOS

In [16]:
df.drop(columns='id').to_excel("Notas_9ano.xlsx",sheet_name="Notas", index=False)